# Implementing Probabilistic Matrix Factorization in PyTorch

From this paper: https://papers.nips.cc/paper/3208-probabilistic-matrix-factorization.pdf.

Data from here: https://grouplens.org/datasets/movielens/

Our goal here is to find feature vectors for our users and movies that allow us to compare them. We start with a big matrix $R$ of movie ratings, with users as rows and movies as columns. Since users don't rate all movies, this is a pretty sparse matrix with a lot of missing values. 

What we can do is factor $R$ into two matrices $U$ and $V$ representing latent feature vectors for users and movies. With these latent features, we can compare users with each other, movies with each other, and predict ratings for movies users haven't seen.

![matrix factorization](matrix_factorization.png)

The big idea behind this paper is that we're going to treat the latent vectors as parameters in a Bayesian model. As a reminder, Bayes theorem:

$$
\large P(\theta \mid D) \propto P(D \mid \theta) P(\theta)
$$

In our model we try to predict the rating matrix $R$ with $U$ and $V$

$$ \large \hat{R} = U^T V$$

In our model we assume the ratings are drawn from a normal distribution with mean $\hat{R}$. What's really cool is that we can place priors on our latent features. 

$$
\begin{aligned}
R &\sim \mathrm{Normal}(U^T V, \sigma^2) \\
U &\sim \mathrm{Normal}(0, \sigma_U^2) \\
V &\sim \mathrm{Normal}(0, \sigma_V^2)
\end{aligned}
$$

The authors of the paper go further and build a hierachical structure for the user vectors

$$
\begin{aligned}
U_i &= Y_i + \frac{\sum_k^M I_{ik}W_k}{\sum_k^M I_{ik}} \\
Y &\sim \mathrm{Normal}(0, \sigma_U^2) \\
W &\sim \mathrm{Normal}(0, \sigma_W^2)
\end{aligned}
$$

With this model, we can maximize the posterior probability with respect to $U$ and $V$, or $V$, $Y$, and $W$ for the hierarchical model. In effect this is just a linear model with fancy regularization.

The authors also do things like converting the ratings to be between 0 and 1, then taking the sigmoid of $U^T V$. For empirical reasons.

In [1]:
import pandas as pd
import torch

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [4]:
ratings.describe()

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


In [5]:
rating_matrix = ratings.pivot(index='userId', columns='movieId', values='rating')
n_users, n_movies = rating_matrix.shape
# Scaling ratings to between 0 and 1, this helps our model by constraining predictions
min_rating, max_rating = ratings['rating'].min(), ratings['rating'].max()
rating_matrix = (rating_matrix - min_rating) / (max_rating - min_rating)

In [6]:
# Replacing missing ratings with -1 so we can filter them out later
rating_matrix[rating_matrix.isnull()] = -1
rating_matrix = torch.FloatTensor(rating_matrix.values)

In [7]:
# This is how we can define our feature matrices
# We're going to be training these, so we'll need gradients
latent_vectors = 5
user_features = torch.randn(n_users, latent_vectors, requires_grad=True)
user_features.data.mul_(0.01)
movie_features = torch.randn(n_movies, latent_vectors, requires_grad=True)
movie_features.data.mul_(0.01)

tensor([[ 0.0067, -0.0105,  0.0074,  0.0099,  0.0122],
        [ 0.0039, -0.0055, -0.0020, -0.0106,  0.0109],
        [ 0.0120, -0.0112, -0.0107,  0.0054, -0.0020],
        ...,
        [-0.0019, -0.0210,  0.0027,  0.0274, -0.0096],
        [ 0.0152, -0.0083,  0.0015, -0.0007, -0.0005],
        [-0.0048,  0.0103, -0.0027,  0.0072,  0.0056]])

In [8]:
class PMFLoss(torch.nn.Module):
    def __init__(self, lam_u=0.3, lam_v=0.3):
        super().__init__()
        self.lam_u = lam_u
        self.lam_v = lam_v
    
    def forward(self, matrix, u_features, v_features):
        non_zero_mask = (matrix != -1).type(torch.FloatTensor)
        predicted = torch.sigmoid(torch.mm(u_features, v_features.t()))
        
        diff = (matrix - predicted)**2
        prediction_error = torch.sum(diff*non_zero_mask)

        u_regularization = self.lam_u * torch.sum(u_features.norm(dim=1))
        v_regularization = self.lam_v * torch.sum(v_features.norm(dim=1))
        
        return prediction_error + u_regularization + v_regularization

In [9]:
criterion = PMFLoss()
loss = criterion(rating_matrix, user_features, movie_features)

In [ ]:
# Actual training loop now

latent_vectors = 30
user_features = torch.randn(n_users, latent_vectors, requires_grad=True)
user_features.data.mul_(0.01)
movie_features = torch.randn(n_movies, latent_vectors, requires_grad=True)
movie_features.data.mul_(0.01)

pmferror = PMFLoss(lam_u=0.05, lam_v=0.05)
optimizer = torch.optim.Adam([user_features, movie_features], lr=0.01)
for step, epoch in enumerate(range(1000)):
    optimizer.zero_grad()
    loss = pmferror(rating_matrix, user_features, movie_features)
    loss.backward()
    optimizer.step()
    if step % 50 == 0:
        print(f"Step {step}, {loss:.3f}")

Step 0, 8252.830
Step 50, 2594.168
Step 100, 1554.992
Step 150, 1133.337
Step 200, 949.591
Step 250, 854.038
Step 300, 793.739
Step 350, 751.326
Step 400, 719.344
Step 450, 695.155
Step 500, 676.232
Step 550, 660.853
Step 600, 648.111
Step 650, 637.387


In [34]:
# Checking if our model can reproduce the true user ratings
user_idx = 7
user_ratings = rating_matrix[user_idx, :]
true_ratings = user_ratings != -1
predictions = torch.sigmoid(torch.mm(user_features[user_idx, :].view(1, -1), movie_features.t()))
predicted_ratings = (predictions.squeeze()[true_ratings]*(max_rating - min_rating) + min_rating).round()
actual_ratings = (user_ratings[true_ratings]*(max_rating - min_rating) + min_rating).round()

print("Predictions: \n", predicted_ratings)
print("Truth: \n", actual_ratings)

Predictions: 
 tensor([4., 2., 4., 4., 3., 5., 3., 4., 5., 3., 3., 4., 2., 4., 4., 3., 4., 3.,
        3., 3., 5., 3., 3., 4., 3., 5., 3., 3., 5., 5., 3., 4., 4., 1., 3., 4.,
        3., 4., 2., 5., 3., 3., 3., 5., 3., 4., 3.], grad_fn=<RoundBackward>)
Truth: 
 tensor([4., 2., 4., 4., 3., 5., 3., 4., 5., 3., 3., 4., 2., 5., 4., 3., 4., 3.,
        3., 3., 5., 3., 3., 4., 3., 5., 3., 3., 5., 5., 3., 4., 5., 1., 3., 4.,
        3., 4., 2., 5., 3., 3., 3., 5., 3., 4., 3.])
